Using the csv outputs of the animal_density we will:
1.Validate population numbers agains FAO STATS of same year
2.Create a data frame of each region specie density % against total population 


1.VAlidation

In [30]:
import pandas as pd

# Function to load a CSV file, select relevant columns, and add a Specie column based on the filename
def load_and_label_csv(file_path, specie_name, columns_to_keep):
    df = pd.read_csv(file_path, usecols=columns_to_keep)
    df['Specie'] = specie_name.capitalize()
    return df

# Define the relevant columns to keep based on buffalo_df
columns_to_keep = ['fid','NAME_1','VARNAME_1','TYPE_1','ENGTYPE_1','NAME_2','VARNAME_2','TYPE_2','ENGTYPE_2','REGION','VARREGION','COUNTRY',
                   'CONTINENT','_sum']  

# Load the CSV files into DataFrames with Specie column, selecting only the relevant columns
cattle_df = load_and_label_csv("cattle_regions_density.csv", 'cattle', columns_to_keep)
buffalo_df = load_and_label_csv("buffalo_regions_density.csv", 'buffalo', columns_to_keep)
sheep_df = load_and_label_csv("sheep_regions_density.csv", 'sheep', columns_to_keep)
goats_df = load_and_label_csv("goats_regions_density.csv", 'goats', columns_to_keep)
pigs_df = load_and_label_csv("pigs_regions_density.csv", 'pigs', columns_to_keep)

# Concatenate all species DataFrames into one
all_species_df = pd.concat([cattle_df, buffalo_df, sheep_df, goats_df, pigs_df])

# Group by 'COUNTRY' and 'Specie' and sum the '_sum' column
zonal_totals = all_species_df.groupby(['COUNTRY', 'Specie'])['_sum'].sum().reset_index()

# Multiply the results by 100 as _sum is the species heads per 1km2 and pixels represent a total of 10km2
zonal_totals['_sum'] = zonal_totals['_sum'] * 100

# Round the results and convert to integers
zonal_totals['_sum'] = zonal_totals['_sum'].round().astype(int)

# Renaming the column for clarity
zonal_totals.rename(columns={'_sum': 'total_sum'}, inplace=True)

print(zonal_totals)



C:\Users\user\AppData\Local\Temp\ipykernel_10884\1124218775.py:5: DtypeWarning: Columns (7,17,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols=columns_to_keep)
C:\Users\user\AppData\Local\Temp\ipykernel_10884\1124218775.py:5: DtypeWarning: Columns (3,7,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols=columns_to_keep)
C:\Users\user\AppData\Local\Temp\ipykernel_10884\1124218775.py:5: DtypeWarning: Columns (7,17,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols=columns_to_keep)
C:\Users\user\AppData\Local\Temp\ipykernel_10884\1124218775.py:5: DtypeWarning: Columns (7,17,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols=columns_to_keep)
C:\Users\user\AppData\Local\Temp\ipykernel_10884\1124218775.py:5: DtypeWarning: C

          COUNTRY   Specie  total_sum
0     Afghanistan  Buffalo       7791
1     Afghanistan   Cattle    7296604
2     Afghanistan    Goats   11353233
3     Afghanistan     Pigs         31
4     Afghanistan    Sheep   19385101
...           ...      ...        ...
1265        Åland  Buffalo          0
1266        Åland   Cattle      12225
1267        Åland    Goats         69
1268        Åland     Pigs          0
1269        Åland    Sheep       4107

[1270 rows x 3 columns]


In [17]:

# FAO totals with COUNTRY and Specie columns
fao_df = pd.read_csv('FAOSTAT_animal_2020_only.csv')

# Ensure that 'COUNTRY' and 'Specie' columns exist and have the correct format in fao_df
fao_df['COUNTRY'] = fao_df['COUNTRY'].astype(str)
fao_df['Specie'] = fao_df['Specie'].str.capitalize()

# Merge zonal_totals and fao_df on 'COUNTRY' and 'Specie' columns
comparison = pd.merge(zonal_totals, fao_df, on=['COUNTRY', 'Specie'], suffixes=('_zonal', '_fao'))

# Print the comparison
comparison



,COUNTRY,Specie,total_sum,Year,Value,Flag Description
0,Afghanistan,Cattle,7296604,2020,5326747,Estimated value
1,Afghanistan,Goats,11353233,2020,7967673,Imputed value
2,Afghanistan,Sheep,19385101,2020,12817638,Estimated value
3,Albania,Buffalo,205,2020,0,Official figure
4,Albania,Cattle,629488,2020,362583,Official figure
...,...,...,...,...,...,...
534,Zambia,Goats,3534846,2020,4042767,Imputed value
535,Zambia,Sheep,310463,2020,206017,Imputed value
536,Zimbabwe,Cattle,7197133,2020,5195154,Official figure
537,Zimbabwe,Goats,6231670,2020,5881701,Imputed value


In [11]:
# Define a function to calculate percentage difference
def percentage_difference(value1, value2):
    if value1 == 0 and value2 == 0:
        return 0  # If both values are zero, consider them to have 0% difference
    try:
        return abs(value1 - value2) / ((value1 + value2) / 2) * 100
    except ZeroDivisionError:
        return float('inf')  # Return infinity if division by zero occurs

# Apply the function to calculate percentage difference and add it as a new column
comparison['percent_diff'] = comparison.apply(lambda row: percentage_difference(row['total_sum'], row['Value']), axis=1)

# Define a threshold for similarity (e.g., 10%)
threshold = 10

# Classify rows based on the threshold
comparison['similar'] = comparison['percent_diff'] <= threshold

# Count the number of similar and different rows
similar_count = comparison['similar'].sum()
different_count = (~comparison['similar']).sum()

# Calculate the percentage of similar and different rows
total_rows = len(comparison)
similar_percentage = (similar_count / total_rows) * 100
different_percentage = (different_count / total_rows) * 100

# Print summary statistics
print(f"Total rows: {total_rows}")
print(f"Similar rows: {similar_count} ({similar_percentage:.2f}%)")
print(f"Different rows: {different_count} ({different_percentage:.2f}%)")

# Optionally, print the first few rows of the comparison DataFrame to inspect
comparison


Total rows: 539
Similar rows: 44 (8.16%)
Different rows: 495 (91.84%)


,COUNTRY,Specie,total_sum,Year,Value,Flag Description,percent_diff,similar
0,Afghanistan,Cattle,7296604,2020,5326747,Estimated value,31.209732,False
1,Afghanistan,Goats,11353233,2020,7967673,Imputed value,35.045562,False
2,Afghanistan,Sheep,19385101,2020,12817638,Estimated value,40.788226,False
3,Albania,Buffalo,205,2020,0,Official figure,200.000000,False
4,Albania,Cattle,629488,2020,362583,Official figure,53.807641,False
...,...,...,...,...,...,...,...,...
534,Zambia,Goats,3534846,2020,4042767,Imputed value,13.405831,False
535,Zambia,Sheep,310463,2020,206017,Imputed value,40.445322,False
536,Zimbabwe,Cattle,7197133,2020,5195154,Official figure,32.310081,False
537,Zimbabwe,Goats,6231670,2020,5881701,Imputed value,5.778226,True


2.Create Data Frame

In [32]:

# Calculate the total population for each species in each country
country_totals = all_species_df.groupby(['COUNTRY', 'Specie'])['_sum'].sum().reset_index()
country_totals.rename(columns={'_sum': 'country_total'}, inplace=True)

# Merge the totals back into the original DataFrame
density_df = pd.merge(all_species_df, country_totals, on=['COUNTRY', 'Specie'])

# Calculate the percentage
density_df['density%'] = (density_df['_sum'] / density_df['country_total']) * 100
# round the percentage column to two decimal places for better readability
density_df['density%'] = density_df['density%'].round(2)
# Round the results and convert to integers
density_df['_sum'] = density_df['_sum'].round().astype(int)
density_df['country_total'] = density_df['country_total'].round().astype(int)



In [34]:

# Renaming the column for clarity
density_df.rename(columns={'fid': 'id','NAME_1': 'admin_level_1_name', 'TYPE_1': 'admin_level_1_type',
                              'ENGTYPE_1': 'admin_level_1', 'VARNAME_1': 'admin_level_1_other_names','NAME_2': 'admin_level_2_name', 'TYPE_2': 'admin_level_2_type',
                              'ENGTYPE_2': 'admin_level_2', 'VARNAME_2': 'admin_level_2_other_names', 'REGION': 'region', 'VARREGION': 'varregion', 'COUNTRY':
                              'country', 'CONTINENT': 'continent', '_sum': 'value', 'Specie': 'specie'}, inplace=True)


In [36]:
#order columns
density_df=density_df[['id', 'continent','country', 'admin_level_1', 'admin_level_1_name', 'admin_level_1_other_names',
       'admin_level_1_type', 'admin_level_2','admin_level_2_name','admin_level_2_other_names', 'admin_level_2_type', 
       'region', 'varregion', 'specie','value', 'country_total', 'density%']]
# Print the first few rows to verify
density_df

,id,continent,country,admin_level_1,admin_level_1_name,admin_level_1_other_names,admin_level_1_type,admin_level_2,admin_level_2_name,admin_level_2_other_names,admin_level_2_type,region,varregion,specie,value,country_total,density%
0,1,Asia,Afghanistan,Province,Badakhshan,Badahšan,Velayat,District,Baharak,NaN,Wuleswali,NaN,NaN,Cattle,669,72966,0.92
1,2,Asia,Afghanistan,Province,Badakhshan,Badahšan,Velayat,District,Darwaz,NaN,Wuleswali,NaN,NaN,Cattle,528,72966,0.72
2,3,Asia,Afghanistan,Province,Badakhshan,Badahšan,Velayat,District,Fayzabad,NaN,Wuleswali,NaN,NaN,Cattle,93,72966,0.13
3,4,Asia,Afghanistan,Province,Badakhshan,Badahšan,Velayat,District,Ishkashim,NaN,Wuleswali,NaN,NaN,Cattle,124,72966,0.17
4,5,Asia,Afghanistan,Province,Badakhshan,Badahšan,Velayat,District,Jurm,NaN,Wuleswali,NaN,NaN,Cattle,470,72966,0.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1782535,356504,Africa,Zimbabwe,Province,Midlands,NaN,Province,District,Zvishavane,NaN,District,NaN,NaN,Pigs,0,3500,0.00
1782536,356505,Africa,Zimbabwe,Province,Midlands,NaN,Province,District,Zvishavane,NaN,District,NaN,NaN,Pigs,0,3500,0.00
1782537,356506,Africa,Zimbabwe,Province,Midlands,NaN,Province,District,Zvishavane,NaN,District,NaN,NaN,Pigs,0,3500,0.00
1782538,356507,Africa,Zimbabwe,Province,Midlands,NaN,Province,District,Zvishavane,NaN,District,NaN,NaN,Pigs,0,3500,0.00


In [37]:
# Save the first few rows of all_species_df to a CSV file
density_df.to_csv('density_df.csv', index=False)